In [1]:
import os
import numpy as np
from spectral_clustering import spectral_clustering
import seaborn as sns
import functions_for_plotting
from asymmetric_laplacian_distribution import get_index_per_class, get_labels, labels_to_layout_mapping
from sklearn.cluster import KMeans
import training_set_split
import seaborn as sns
import prediction_strength
import importlib
import matplotlib.pyplot as plt
from prediction_strength import get_F1_score_per_k
from matplotlib.legend import Legend
from training_set_split import get_training_folds
import wagenaar_dataset

/Users/paule/anaconda3/envs/bon17/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.nearest_centroid module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Spectral Clustering

Clustering has the goal to divide data points into several meaningful groups such
that points in the same group are similar and points in different groups are dissimilar to each other. These groups should afterwards reveal and give an intuiton of the data structure. 
Clustering belongs to the so called unsupervised-learning problems for which no underlying correct answer exists. While there is no correct answer there are still ways to define good or better and bad or worse solution and therefore also better or worse performing algorithms to solve this problem. 
Beside simple and widely used algorithms like K-Means Clustering or Gaussian Mixture Models (GMM), Spectral Clustering turned out to be one of the good performing clustering algorithms.  


Spectral Clustering was developed and popularized by Shi & Malik (2000) and  Ng, Jordan, & Weiss (2002). It is based on spectal graph theory which studies the properties of graphs by using associated matrices like the adjacency or the graph Laplacian matrix.
The idea behind spectral clustering is to reformulate the problem of clustering in terms of graph theory by constructing a similarity graphs and group the graph into subgraphs that are most similar within and dissimilar to each other.  
In order to achieve this the algorthim must come up with a partition 'such that the edges between different groups have very low weights (which means that points in different clusters are dissimilar from each other) and the edges within a group have high weights (which means that points within the same cluster are similar to each other)' (von Luxburg, 2006).  
Therefore the first step of this approach is to calculate some notion of similarity between each data point and construct a similarity graph. Each vertex in the graph corresponds to a datapoint in the dataset. The connection between vertices represent the similarity between datapoints.   
Having constructed the similarity graph the problem of finding similar groups can be reformulated by the so called Min-Cut problem (Mohar, 1991) in which we try to cut the graph into subgraphs by as few edges or with as possible. However this simple approach can not account for outliers. Outliers in a graph are represented as nodes connected to the rest of the graph by an single edge (see Fig 1.). Cutting off those single nodes would lead to highly inbalanced splits. Therefore a better formulation for our grouping problem is the so called RatioCut (Hagen and Kahng, 1992) which adds the constraint of balancing the resulting parts in terms of number of vertices in each subgraph.  
Minimizing the RatioCut problem however was found out to be a NP-hard problem (Wagner & Wagner, 1993) but a relaxed version of the ratio cut was shown to be solveable by the usage of the so called Graph-Laplacian and the Rayleigh-Ritz theorem. Mohar (1997) could show that the eigenvectors corresponding to the smallest eigenvalues of a graph Laplacian can be related to the number of connected components in the graph.  

The Graph Lablacian is defined by:  $L = D - W$ with $D = diag(d_1,...d_n)$ beeing teh diagonal degree matrix of weighted degrees $d_i = \sum_{j=1}^{n}w_{ij}$ for each node and W being the weight matrix of the similarity graph.  

The Cluster structure of the data can be preserved by projecting the data into a lower dimensional space using the first k eigenvectors. This results in the so called spectral-embedding in which the points of corresponding connected components are well enough embedded to be easily clustered by e.g. K-Means. 
Spectral Clustering therefore performs dimensionality reduction using the eigenvectors of a graph laplacian  before clustering the data in the transformed space.
Although Spectral Clustering only solves a relaxed version of the inital problem and does not have a approximation guarantee (Guattery & Miller, 1998) it shows good results in practice. The results obtained
by spectral clustering often outperform traditional approaches. Furthermore the fact that spectral clustering formulates a standard linear algebra problem which can be efficiently solved by standard linear algebra methods makes it popular. 


## Similarity Graph
We started the analysis with a vanilla version of the Spectral Clustering method. We constructed a similarity graph using the pairwise euclidean distance. For each datapoint we calculated the k=10 nearest neighbours and connected corresponding nodes by an edge with weight 1. This led to the symmetric adjacency matrix $A$ beeing equal to the weight matrix $W$. As a next step we calculated the degree for each node and constructed the resulting graph laplacian $L$ with $L = D - A$. 
Although in some cases regularization seem to improve results especially in cases where the similarity graph contains so called k-dangling sets (Zhang and Rohe, 2018) we did not make us of it. Regularization with respect to Zhang and Rohe (Zhang and Rohe, 2018) can be applied to the weight matrix $W$ by adding a small weight to each edge  $\tilde{W}= W + \frac{\tau}{n} \cdot J$ where $J$ is the all-ones-matrix and $\tau$ a small parameter.



## Graph-Laplacian
With respect to von Luxburg 2008 we normalized the laplacian by $L_norm = D^{-\frac{1}{2}}(D - A)D^{-\frac{1}{2}}$ (Ng,Jordan & Weiss,2002). While the standard laplacian is derived from the relaxes version of the RatioCut, the normalized follows from the relaxation of the so called NCut problem (Shiand Malik, 2000; Ding, 2004). Similar to the RatioCut the NCut problem includes a balancing condition. However, instead of balancing the size of each subgraph measured as its number of vertices the size in NCut is measured by the weights of all edges in a subgrpah also known as the volume. The volume of a subgraph A is defined as $vol(A):=\sum_{i\in A}d_i$ with $d_i$ beeing the degree of node $i$.
The normalized laplacian derived from the relaxed NCut problem therefore not only consider size of clusters but also its connectivity which reffering again to von Luxburg 2008 tend to give better results in practice.


## Clustering 
We proceded with the normalized spectral clustering according to Ng, Jordan, and Weiss (Ng,Jordan & Weiss,2002). Having computed the normalized laplacian we calculated the corresponding eigenvectors and eigenvalues. Using the first k eigenvectors corresponding to the k smallest eigenvalues we constructed the matrix $U \in \mathbb{R}^{n\times k}$ containing the eigenvectors $u_1,\cdots,u_k$ as columns. We formed the matrix $T \in \mathbb{R}^{n\times k}$ by normalizing the rows of $U$ to norm 1. 
Each datapoint $x_i$ is now embedded in the k dimensional space of row normalized eigenvectors as $y_i \in \mathbb{R}^k$. As a last step we cluster the points $y_i$ $i=1,\cdots,n$ with the k-means algorithm into clusters $C1,\cdots,Ck$.



## Fig. 1 Min-Cut Problem
<img src="mincut_problem.png" width="400" height="400" align="left"/>

# Dataset
- load dataset
- load culture dict specifiying for each culture start and end point with respect to the dataset for indexing

In [2]:
data_dir = "data/raw_data/daily_spontanous_dense/culture2_over_days/"
data = np.load(data_dir + "data_burst_by_time_culture_2_2.npy").T
culture_dict = np.load(data_dir + "culture_dict_culture_2_2.npy", allow_pickle=True).item()

# Cluster Full Dataset

## Spectral Clustering Parameter

In [3]:
metric = "euclidean"
metric_type = "distance"
range_clusters = range(1,101)
k=10
reg=None
mutual = False
weighting = False
normalize = True

In [8]:
labels, eigvec, eigval = spectral_clustering(data, metric, metric_type, range_clusters, k=k, mutual = mutual, weighting = weighting, normalize = normalize, reg_lambda = reg, save_laplacian = False, save_eigenvalues_and_vectors = False)

Calculate euclidean matrix for constructing KNN-Graph
Build symmetric KNN-Graph based on Distance of data points!
Weighting: False
Calculate Normalized Laplacians
Normalization: symmetric
Calculate Eigenvalues and Vectors of Laplacian


In [9]:
#np.save("labels_culture_2_2_Euclidean_k=10_reg=none_100clusters" ,labels)
#np.save("eigval_culture_2_2_Euclidean_k=10_reg=none_100clusters" ,eigval)

# Cluster Splitted Dataset

In [10]:
def cluster_data_splits(data, train_fold_indices, valid_fold_indices,metric = "euclidean",metric_type="distance", range_clusters = range(1,50),k=10, reg=None, train_only = False,mutual=False,weighting=False, normalize = True):
    print("Start clustering training folds!")
    train_fold_labels = []
    train_fold_eigenvalues = []
    for fold_indices in train_fold_indices:
        train_data = data[fold_indices]
        labels, eigvec, eigval = spectral_clustering(train_data, metric, metric_type, range_clusters, k=k, mutual = mutual, weighting = weighting, normalize = normalize, reg_lambda = reg, save_laplacian = False, save_eigenvalues_and_vectors = False)
        train_fold_labels.append(labels)
        train_fold_eigenvalues.append(eigval)
    print("Done...")
    if train_only:
        return train_fold_labels, train_fold_eigenvalues
    
    else:
        print("Start clustering validation folds!")
        valid_fold_labels = []
        valid_fold_eigenvalues = []
        for fold_indices in valid_fold_indices:
            valid_data = data[fold_indices]
            labels, eigvec, eigval = spectral_clustering(valid_data, metric, metric_type, range_clusters, k=k, mutual = mutual, weighting = weighting, normalize = normalize, reg_lambda = reg, save_laplacian = False, save_eigenvalues_and_vectors = False)
            valid_fold_labels.append(labels)
            valid_fold_eigenvalues.append(eigval)
        print("Done...")
        return train_fold_labels, train_fold_eigenvalues, valid_fold_labels, valid_fold_eigenvalues

## Specify Data Splitting
split styles: 
- 'balanced' with respect to cultures (5 fold random split for each culture) --> culture_dict must be provided
- 'random' 5 fold random split --> no culture_dict needed
- 'unbalanced' some clusters only occure in training fold not in validation fold --> culture_dict and only_training_clusters (list of cluster per fold which should only be occur in training data) 

In [4]:
split_style = "balanced"
folds = 5

In [5]:
train_fold_indices, valid_fold_indices = training_set_split.get_training_folds(data,culture_dict,cluster_split = split_style,folds = folds)

## Cluster Folds

### Spectral Clustering Parameter

In [ ]:
metric = "euclidean"
metric_type = "distance"
range_clusters = range(1,101)
k=10
reg=None
mutual = False
weighting = False
normalize = True

In [20]:
train_fold_labels, train_fold_eigenvalues, valid_fold_labels, valid_fold_eigenvalues = cluster_data_splits(data, train_fold_indices, valid_fold_indices,metric = metric,metric_type=metric_type,range_clusters = range_clusters,k=k, reg=reg, train_only = False)

Start clustering training folds!
Calculate euclidean matrix for constructing KNN-Graph
Build symmetric KNN-Graph based on Distance of data points!
Weighting: False
Calculate Normalized Laplacians
Normalization: symmetric
Calculate Eigenvalues and Vectors of Laplacian
Calculate euclidean matrix for constructing KNN-Graph
Build symmetric KNN-Graph based on Distance of data points!
Weighting: False
Calculate Normalized Laplacians
Normalization: symmetric
Calculate Eigenvalues and Vectors of Laplacian
Calculate euclidean matrix for constructing KNN-Graph
Build symmetric KNN-Graph based on Distance of data points!
Weighting: False
Calculate Normalized Laplacians
Normalization: symmetric
Calculate Eigenvalues and Vectors of Laplacian
Calculate euclidean matrix for constructing KNN-Graph
Build symmetric KNN-Graph based on Distance of data points!
Weighting: False
Calculate Normalized Laplacians
Normalization: symmetric
Calculate Eigenvalues and Vectors of Laplacian
Calculate euclidean matrix 

In [21]:
#np.save("labels_culture_2_2_Euclidean_k=10_reg=None_5_fold_balanced_train_100clusters.npy" ,train_fold_labels)
#np.save("eigval_culture_2_2_Euclidean_k=10_reg=None_5_fold_balanced_train_100clusters.npy" ,train_fold_eigenvalues)

In [22]:
#np.save("labels_culture_2_2_Euclidean_k=10_reg=None_5_fold_balanced_valid_100clusters.npy" ,valid_fold_labels)
#np.save("eigval_culture_2_2_Euclidean_k=10_reg=None_5_fold_balanced_valid_100clusters.npy" ,valid_fold_eigenvalues)